### Starting with Data analysis of new charging stations data

In [2]:
import pandas as pd

# Load your new dataset
df_stations = pd.read_csv("chargingStations.csv")

# Basic info
print("SHAPE:", df_stations.shape)
print("\nCOLUMNS:\n", df_stations.columns.tolist())
print("\nSUMMARY STATS:\n", df_stations.describe(include='all'))
print("\nNULL VALUES:\n", df_stations.isnull().sum())


SHAPE: (4777, 98)

COLUMNS:
 ['chargeDeviceID', 'reference', 'name', 'latitude', 'longitude', 'postcode', 'countryCode', 'deviceNetworks', 'locationType', 'connector1ID', 'connector1Type', 'connector1RatedOutputKW', 'connector1OutputCurrent', 'connector1RatedVoltage', 'connector1ChargeMethod', 'connector1ChargeMode', 'connector1TetheredCable', 'connector1Status', 'connector1Description', 'connector1Validated', 'connector2ID', 'connector2Type', 'connector2RatedOutputKW', 'connector2OutputCurrent', 'connector2RatedVoltage', 'connector2ChargeMethod', 'connector2ChargeMode', 'connector2TetheredCable', 'connector2Status', 'connector2Description', 'connector2Validated', 'connector3ID', 'connector3Type', 'connector3RatedOutputKW', 'connector3OutputCurrent', 'connector3RatedVoltage', 'connector3ChargeMethod', 'connector3ChargeMode', 'connector3TetheredCable', 'connector3Status', 'connector3Description', 'connector3Validated', 'connector4ID', 'connector4Type', 'connector4RatedOutputKW', 'connec

### For ChargePoint Network Bv Netherlands (might change it to qwello) 20 values

In [9]:
# Filter rows where device network matches
filtered = df_stations[df_stations['deviceNetworks'] == "ChargePoint Network (Netherlands) B.V."]

# Select only specific columns
result = filtered[["town", "county", "deviceOwnerName", "attribution","borough"]]

# Display the result
print(result)

        town          county deviceOwnerName  \
4151  London  Greater London       *No Owner   
4152  London  Greater London       *No Owner   
4153  London  Greater London       *No Owner   
4154  London  Greater London       *No Owner   
4155  London  Greater London       *No Owner   
4156  London  Greater London       *No Owner   
4157  London  Greater London       *No Owner   
4158  London  Greater London       *No Owner   
4159  London  Greater London       *No Owner   
4160  London  Greater London       *No Owner   
4161  London  Greater London       *No Owner   
4162  London  Greater London       *No Owner   
4163  London  Greater London       *No Owner   
4164  London  Greater London       *No Owner   
4165  London  Greater London       *No Owner   
4166  London  Greater London       *No Owner   
4167  London  Greater London       *No Owner   
4168  London  Greater London       *No Owner   
4169  London  Greater London       *No Owner   
4170  London  Greater London       *No O

### For Drax Energy Solutions Limited 6 values

In [10]:
# Filter rows where device network matches
filtered = df_stations[df_stations['deviceNetworks'] == "Drax Energy Solutions Limited"]

# Select only specific columns
result = filtered[["town", "county", "deviceOwnerName", "attribution","borough"]]

# Display the result
print(result)

          town          county        deviceOwnerName  \
2597    London  Greater London  Ista Energy Solutions   
2598    London  Greater London  Ista Energy Solutions   
2599    London  Greater London  Ista Energy Solutions   
2600  Bankside  Greater London  Ista Energy Solutions   
2601   London          London   Ista Energy Solutions   
2602   London          London   Ista Energy Solutions   

                        attribution                 borough  
2597  Drax Energy Solutions Limited  hammersmith and fulham  
2598  Drax Energy Solutions Limited  hammersmith and fulham  
2599  Drax Energy Solutions Limited                 lambeth  
2600  Drax Energy Solutions Limited                 lambeth  
2601  Drax Energy Solutions Limited                 lambeth  
2602  Drax Energy Solutions Limited                 lambeth  


### For E.ON Drive its camden so whatever chargers are most in camden we will change the values to that.

In [ ]:
# Filter rows where device network matches
filtered = df_stations[df_stations['deviceNetworks'] == "E.ON Drive"]

# Select only specific columns
result = filtered[["town", "county", "deviceOwnerName", "attribution","borough"]]

# Display the result
print(result)

               town  county deviceOwnerName attribution borough
2333  Charing Cross  London      E.ON Drive  E.ON Drive  camden


### For London Borough of Hackney	tower hamlets whatever is highest

In [13]:
# Filter rows where device network matches
filtered = df_stations[df_stations['deviceNetworks'] == "London Borough of Hackney"]

# Select only specific columns
result = filtered[["town", "county", "deviceOwnerName", "attribution","borough"]]

# Display the result
print(result)

             town          county      deviceOwnerName  \
67  Bethnal Green  Greater London  Equans EV Solutions   

                  attribution        borough  
67  London Borough of Hackney  tower hamlets  


### Deleting the charging stations where their pricings couldnt be found

In [ ]:
# Define networks to remove
networks_to_remove = ["London Borough of Hackney", "E.ON Drive","Drax Energy Solutions Limited","ChargePoint Network (Netherlands) B.V."]

# Filter out rows with those device networks
df_stations_cleaned = df_stations[~df_stations["deviceNetworks"].isin(networks_to_remove)].copy()

# Save the cleaned dataset
df_stations_cleaned.to_csv("chargingStations.csv", index=False)

### As per online information Charge Your Car and BP Pulse Polar are both BP Pulse now

In [ ]:
# Replace specified names
df_stations['deviceNetworks'] = df_stations['deviceNetworks'].replace({
    "Charge Your Car": "BP Pulse",
    "BP-Pulse (POLAR)": "BP Pulse"
})

# Save the updated dataset
df_stations.to_csv("chargingStations.csv", index=False)

### Shell Recharge solutions was found to be a private charging so we are removing them

In [ ]:
# Define networks to remove
networks_to_remove = ["Shell Recharge Solutions"]

# Filter out rows with those device networks
df_stations_cleaned = df_stations[~df_stations["deviceNetworks"].isin(networks_to_remove)].copy()

# Save the cleaned dataset
df_stations_cleaned.to_csv("chargingStations.csv", index=False)

### Now lets clean the dataset to get our desired columns

In [8]:
df = pd.read_csv("chargingStations.csv")

In [4]:
# Drop all columns where *every* value is NaN
df_cleaned = df.dropna(axis=1, how='all')
dropped_cols = df.columns.difference(df_cleaned.columns)
print("Dropped empty columns:", dropped_cols.tolist())

Dropped empty columns: ['connector1Description', 'connector2Description', 'connector3Description', 'connector4Description', 'connector5Description', 'connector6Description', 'connector7Description', 'connector8Description']


### Removing redundant or not needed columns from dataset regarding connectors

In [9]:
connector_cols_to_keep = []

for i in range(1, 9):
    connector_cols_to_keep += [
        f'connector{i}Type',
        f'connector{i}RatedOutputKW',
        f'connector{i}ChargeMethod',
        f'connector{i}Status'
    ]

# Get all connector-related columns
all_connector_cols = [col for col in df.columns if col.startswith('connector')]

# Drop only those not in the keep list
connector_cols_to_drop = list(set(all_connector_cols) - set(connector_cols_to_keep))

# Drop and save
df.drop(columns=connector_cols_to_drop, inplace=True)
df.to_csv("chargingStations.csv", index=False)
